# Исследование надежности заемщиков


Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку. 



## Общая информация о данных

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('.../datasets/data.csv')

**Выведем первые 20 строчек датафрейма `data` на экран.**

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Выведем основную информацию о датафрейме с помощью метода `info()`.**

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

**Выведем количество пропущенных значений для каждого столбца. Используем комбинацию двух методов.**

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце обработаем на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.**

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

**В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработаем значения в этом столбце: заменим все отрицательные значения положительными с помощью метода `abs()`.**

In [6]:
data['days_employed'] = data['days_employed'].abs()

**Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.**

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.

**Выведем перечень уникальных значений столбца `children`.**

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

**В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков

**Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**Убедимся, что все пропуски заполнены. Проверим себя и ещё раз выведите количество пропущенных значений для каждого столбца с помощью двух методов.**

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

**Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

**Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру. Проверим остальные столбцы.**

In [14]:
data['education'] = data['education'].str.lower()

**Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их.**

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

**На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используем собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Задание 17. Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Задание 18. Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Используем собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучим данные в столбце `purpose` и определим, какие подстроки помогут вам правильно определить категорию.**

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследуем данные и ответим на вопросы

Переименуем столбцы исходного датафрейма:

In [22]:
data = data.rename(columns = {'children':'Количество детей', "family_status":"Семейное положение", 
                              "total_income_category": "Категория уровня дохода", "purpose_category": "Цель кредита"
                             })

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

Оценим наличие задолженности у клиетов с разным количеством детей. Для этого создадим сводную таблицу, где рассмотрим группы клиентов по количеству детей и посмотрим для каждой группы общее количество клиентов, количество клиентов, имеющих задолженность, а также найдем какой процент от общего числа клиентов составляют клиенты с задолженностью для каждой группы:

In [23]:
data_children = data.pivot_table(index = 'Количество детей', values = 'debt', aggfunc = {'count','sum','mean'})
data_children['mean'] = data_children['mean']*100
data_children['count'] = data_children['count'].astype(int)
data_children['sum'] = data_children['sum'].astype(int)
data_children = data_children.rename(columns = {'count':'Всего клиентов', 'sum':'Клиентов с задолженностью', 'mean':'Процент клиентов с задолженностью'})
data_children_final = pd.DataFrame(data = data_children, columns = ['Всего клиентов','Клиентов с задолженностью','Процент клиентов с задолженностью'])
display(data_children_final)

,Всего клиентов,Клиентов с задолженностью,Процент клиентов с задолженностью
Количество детей,,,
0,14091,1063,7.543822
1,4808,444,9.234609
2,2052,194,9.454191
3,330,27,8.181818
4,41,4,9.756098
5,9,0,0.000000


**Вывод:** анализируя полученную таблицу мы можем сделать вывод о том, что количество клиентов, имеющих задолженность ниже среди клиентов без детей и примерно равно для групп клиентов с разным количеством детей, исключение составляют только клиенты с 5 детьми, но их количество очень мало по сравнению с другими группами и общим количеством клиентов, поэтому данные по этой группе не являются репрезентативными. Из этого мы можем сделать вывод, что количество детей не влияет на возврат кредита в срок, однако среди людей без детей процент должников ниже.

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

Оценим наличие задолженности у клиетов с разным семейным положением. Для этого создадим сводную таблицу, где рассморим клиентов по семейному положению и посмотрим для каждой группы общее количество клиентов, количество клиентов, имеющих задолженность, а также найдем какой процент от общего числа клиентов составляют клиенты с задолженностью для каждой группы:

In [24]:
data_family = data.pivot_table(index = 'Семейное положение', values = 'debt', aggfunc = {'count','sum','mean'})
data_family['mean'] = data_family['mean']*100
data_family['count'] = data_family['count'].astype(int)
data_family['sum'] = data_family['sum'].astype(int)
data_family = data_family.rename(columns = {'count':'Всего клиентов', 'sum':'Клиентов с задолженностью', 'mean':'Процент клиентов с задолженностью'})
data_family_final = pd.DataFrame(data = data_family, columns = ['Всего клиентов','Клиентов с задолженностью','Процент клиентов с задолженностью'])
display(data_family_final)

,Всего клиентов,Клиентов с задолженностью,Процент клиентов с задолженностью
Семейное положение,,,
Не женат / не замужем,2796,273,9.763948
в разводе,1189,84,7.064760
вдовец / вдова,951,63,6.624606
гражданский брак,4134,385,9.313014
женат / замужем,12261,927,7.560558


**Вывод:** анализируя полученную таблицу мы можем сделать вывод о том, что количество клиентов, имеющих задолженность ниже среди вдов/вдовцов, а также среди людей в разводе. Больше всего имеют задолженность не женатые и состоящие в гражданском браке. Из этого мы можем сделать вывод, что семейное положение влияет на возврат кредита в срок.

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Оценим наличие задолженности у клиетов с разным уровнем дохода. Для этого создадим сводную таблицу, где рассмотрим клиентов по уровню дохода (клиенты были разделены на категории по уровню дохода в пункте 2.6, будем базироваться на этих категориях) и посмотрим для каждой группы общее количество клиентов, количество клиентов, имеющих задолженность, а также найдем какой процент от общего числа клиентов составляют клиенты с задолженностью для каждой группы:

In [25]:
data_income = data.pivot_table(index = 'Категория уровня дохода', values = 'debt', aggfunc = {'count','sum','mean'})
data_income['mean'] = data_income['mean']*100
data_income['count'] = data_income['count'].astype(int)
data_income['sum'] = data_income['sum'].astype(int)
data_income = data_income.rename(columns = {'count':'Всего клиентов', 'sum':'Клиентов с задолженностью', 'mean':'Процент клиентов с задолженностью'})
data_income_final = pd.DataFrame(data = data_income, columns = ['Всего клиентов','Клиентов с задолженностью','Процент клиентов с задолженностью'])
display(data_income_final)

,Всего клиентов,Клиентов с задолженностью,Процент клиентов с задолженностью
Категория уровня дохода,,,
A,25,2,8.000000
B,5014,354,7.060231
C,15921,1353,8.498210
D,349,21,6.017192
E,22,2,9.090909


**Вывод:** анализируя полученную таблицу мы можем сделать вывод о том, что количество клиентов, имеющих задолженность ниже среди людей с маленьким доходом (категория "D": доход от 30 до 50 тыс.). Чаще имеют задолженность люди со средним доходом (категория "С": доход от 50 до 200 тыс.). Однако эта категория является самой многочисленной, поэтому справедливо было бы ее сравнивать лишь с категорией примерно одного порядка по численности клиентов - категорией "В" (доход от 200 тыс. до 1 млн), а остальными группами ("Е" и "А" - с самыми низкими и высокими доходами) принебречь, т.к. количество клиентов в этих категориях существенно меньше по сравнению с другими категориями и общим количеством клиентов, поэтому данные по этим категориям не являются репрезентативными. В категории "В" процент должников ниже примерно на 1,5% чем в категории "С". Из этого мы можем сделать вывод, что доход влияет на возврат кредита в срок.

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

Оценим наличие задолженности у клиетов с разным целями кредита. Для этого создадим сводную таблицу, где рассмотрим клиентов по целям кредита (цели кредита были разделены на категории в пункте 2.6, будем базироваться на этих категориях) и посмотрим для каждой группы общее количество клиентов, количество клиентов, имеющих задолженность, а также найдем какой процент от общего числа клиентов составляют клиенты с задолженностью для каждой группы:

In [26]:
data_purpose = data.pivot_table(index = 'Цель кредита', values = 'debt', aggfunc = {'count','sum','mean'})
data_purpose['mean'] = data_purpose['mean']*100
data_purpose['count'] = data_purpose['count'].astype(int)
data_purpose['sum'] = data_purpose['sum'].astype(int)
data_purpose = data_purpose.rename(columns = {'count':'Всего клиентов', 'sum':'Клиентов с задолженностью', 'mean':'Процент клиентов с задолженностью'})
data_purpose_final = pd.DataFrame(data = data_purpose, columns = ['Всего клиентов','Клиентов с задолженностью','Процент клиентов с задолженностью'])
display(data_purpose_final)

,Всего клиентов,Клиентов с задолженностью,Процент клиентов с задолженностью
Цель кредита,,,
операции с автомобилем,4279,400,9.347978
операции с недвижимостью,10751,780,7.255139
получение образования,3988,369,9.252758
проведение свадьбы,2313,183,7.911803


**Вывод:** анализируя полученную таблицу мы можем сделать вывод о том, что количество клиентов, имеющих задолженность ниже среди тех, кто берет кредит на операции с недвижимостью, а также на проведение свадьбы. Больше всего имеют задолженность те, кто берет кредит на операции с автомобилем и получение образования. Из этого мы можем сделать вывод, что цель кредита влияет на возврат кредита в срок.

#### 3.5 Приведите возможные причины появления пропусков в исходных данных.

В пункте 2.1 мы определили, что пропуски содержатся в 2 столбцах таблицы: 'days_employed' и 'total_income'. Выведем на экран по 25 строк исходной таблицы, содержащей пропуски в этих столбцах и попробуем найти совпадающие значения в других столбцах(например, проверим, что пропуски как-то связаны с типом занятости или каким-то другим столбцом)

In [27]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')
display(data.loc[data['days_employed'].isna()].head(25))
display(data.loc[data['total_income'].isna()].tail(25))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21271,2,NaN,42,среднее,1,гражданский брак,1,M,сотрудник,1,NaN,операции со своей недвижимостью
21281,1,NaN,30,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,покупка коммерческой недвижимости
21294,0,NaN,45,ВЫСШЕЕ,0,женат / замужем,0,F,сотрудник,0,NaN,высшее образование
21300,2,NaN,45,Среднее,1,гражданский брак,1,M,компаньон,0,NaN,получение дополнительного образования
21305,0,NaN,59,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство собственной недвижимости
21308,0,NaN,48,среднее,1,женат / замужем,0,F,компаньон,0,NaN,покупка жилья для сдачи
21311,0,NaN,49,среднее,1,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для сдачи
21321,0,NaN,56,Среднее,1,женат / замужем,0,F,пенсионер,0,NaN,операции с недвижимостью
21350,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,на покупку автомобиля
21364,0,NaN,50,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,операции со своей недвижимостью


Как мы можем заметить при выводе строк с пропущенными значениями в столбце 'days_employed' значения в 'total_income' также пропущены и наоборот. Зная количество пропусков для каждого из столбцов (найдены в пункте 2.1) проверим, совпадает ли это количество с числом строк имеющих одновременно пропуски сразу в двух столбцах:

In [28]:
print(len(data.loc[(data['days_employed'].isna()) & (data['total_income'].isna())]))

2174


Полученное значение пропусков равно значению пропусков в каждом из столбцов, следовательно все строки с пропущенными значениями в 'days_employed' имеют и пропуски в 'total_income'.

*Ответ:* анализируя полученные данные мы можем сделать вывод о том, что пропуски в столбцах 'days_employed' и 'total_income' никак не коррелируются с другими столбцами, однако коллерируются между собой - строки с пропущенными значениями в 'days_employed' имеют и пропуски в 'total_income'. Пропуски могли возникнуть по техническим причинам - из-за потери части данных(например, автоматизированная система сбора информации дала сбой) или на этапе загрузки/выгрузки данных была потеряна часть информации, а также пропуски могли возникнуть по причине человеческого фактора - информацию могли не требовать в определенных случаях или клиент сам мог не предоставлять данные. А также могли быть допущены ошибки при формировании таблиц.

#### 3.6 Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:* в пункте 2.1 мы нашли количество пропущенных значений - оно примерно равно 10% от общего количества значений в данных столбцах, поэтому удалить строки с пропусками было бы не совсем верным решением. Следовательно необходимо было заполнить пропуски характерными значениями - средними или медианными значениями. Медианные значения были выбраны, т.к. мы имеем некоторые значения сильно выделяющиеся среди большинства и использование среднего было бы менее корректно.

### Шаг 4: общий вывод.

Исследовав датафрейм с данными о клиентах банка мы можем сделать следующие выводы:
1) Клиенты без детей детей реже имеют задолженность, чем клиенты с детьми, однако среди клиентов с разным количеством детей мы имеем примерно одинаковый процент дожников;\
2) Количество клиентов, имеющих задолженность ниже среди вдов/вдовцов, а также среди людей в разводе. Больше всего имеют задолженность не женатые и состоящие в гражданском браке;\
3) Клиенты со средним доходом чаще имеют задолженность, чем клиенты с высоким доходом, однако количество клиентов со средним доходом в разы выше количества клиентов во всех остальных группах;\
4) Клиенты, берущие кредит для операций с недвижимостью и проведения свадеб, реже имеют задолженность, чем те, кто берет кредит на авто и образование.\
С точки зрения банка идеальным клиеном был бы вдовец/вдова без детей, <b> с высоким доходом (от 200 тыс. до 1 млн.) </b>, который берет кредит на операцию с недвижимостью. А самым рисковым - не женатый/не замужний клиент с детьми и <b>средним доходом (от 50 до 200 тыс.)</b>, который берет кредит на автомобиль.